# Visualize Networks

In [1]:
import graph_tool.all as gt
import graph_tool.draw as gtd
from graph_tool.stats import vertex_hist
import numpy as np

In [2]:
# Load the graph
g = gt.load_graph("../../../Data/Daily_graphs/2021-04-28.graphml")
g

<Graph object, directed, with 624358 vertices and 460845 edges, 3 internal vertex properties, 1 internal edge property, at 0x7f4b27ea4f10>

In [3]:
# Layout
pos_fruchterman_reingold = gtd.fruchterman_reingold_layout(g)
pos_sfdp = gtd.sfdp_layout(g)

In [ ]:
# Assign colors based on 'Political Affiliation'
color_dict = {
    'No Retweets': [0.5, 0.5, 0.5, 1],        # Gray for 'No Retweets'
    'Retweets Centro': [0, 1, 0, 1],          # Green for 'Retweets Centro'
    'Retweets Derecha': [1, 0, 0, 1],         # Red for 'Retweets Derecha'
    'Retweets Izquierda': [0, 0, 1, 1],       # Blue for 'Retweets Izquierda'
    'nan': [0.5, 0.5, 0.5, 1],                # Gray for 'nan'
}

In [ ]:
# Get vertex colors from the dictionary
vertex_colors = g.new_vertex_property('vector<double>')
for v in g.vertices():
    affiliation = g.vertex_properties["Political Affiliation"][v]
    vertex_colors[v] = color_dict.get(affiliation, [0.5, 0.5, 0.5, 1])  # Default to gray if not found

# Node Size based on Degree (influenced by edge weights)
weight = g.edge_properties["weight"]
hist, _ = vertex_hist(g, "total", weight=weight)
max_size = 25
min_size = 5
vertex_sizes = g.new_vertex_property('double')
size_factor = (hist - hist.min()) / (hist.max() - hist.min())
vertex_sizes.a = size_factor * (max_size - min_size) + min_size

In [ ]:
# Draw the graph
gtd.graph_draw(g, pos_fruchterman_reingold, vertex_fill_color = vertex_colors, vertex_size = vertex_sizes, output_size = (1000, 1000))